In [7]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import numpy as np

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

PRETRAIN_MODEL = '/home/james/models/mistralai/Mistral-7B-Instruct-v0.3'
tokenizer = AutoTokenizer.from_pretrained(PRETRAIN_MODEL, quantization_config=quant_config)
model = AutoModelForCausalLM.from_pretrained(PRETRAIN_MODEL, quantization_config=quant_config)

def generate_embedding(document):
    inputs = tokenizer(document, return_tensors='pt')
    # print("Tokenized inputs:", inputs)
    embeddings = []
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings.append(outputs.logits.mean(dim=1).cpu().numpy())

    # embedding = outputs.last_hidden_state[0, 0, :].numpy()
    # print("Generated embedding:", embedding)
    return embeddings

text1 = "this is a test"
text2 = "this is another test"
text3 = "there are other tests"

embedding1 = generate_embedding(text1)
embedding2 = generate_embedding(text2)
embedding3 = generate_embedding(text3)

are_equal = np.array_equal(embedding1, embedding2) and np.array_equal(embedding2, embedding3)
if are_equal:
    print("The embeddings are the same.")
else:
    print("The embeddings are not the same.")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


The embeddings are not the same.


In [6]:
embedding1

[array([[-7.834375 , -7.7234373,  0.4879486, ..., -6.6046877, -5.966797 ,
         -4.4867187]], dtype=float32)]